In [24]:
import requests #helps to fetch data from API
import pandas as pd #for handling and analysing data
import numpy as np #for numerical operations
from sklearn.model_selection import train_test_split #to split data into training and testing sets
from sklearn.preprocessing import LabelEncoder #to convert catogerical data into numerical values
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor #models for classification and regression
from sklearn.metrics import mean_squared_error #to measure the accuracy of our predictions
from datetime import datetime, timedelta # to handle date and time
import pytz

In [27]:
API_KEY = '62938e7d4a341f8b07802f6d4a6d5521'
BASE_URL = 'https://api.openweathermap.org/data/2.5/weather?lat=44.34&lon=10.99&appid=62938e7d4a341f8b07802f6d4a6d5521'

In [29]:
def get_current_weather(city):
  url = f"{BASE_URL}weather?q={city}&appid={API_KEY}&units=metric" #contruct the API  request URL
  response = requests.get(url) #send the get request to API
  data = response.json()

  # Debug print to inspect response
  print("API Response:", data)

  # Handle failed API responses
  if response.status_code != 200 or 'name' not in data:
      print("Error fetching data:", data.get('message', 'Unknown error'))
      return None
  
  return {
      'city': data['name'],
      'current_temp': round(data['main']['temp']),
      'feels_like': round(data['main']['feels_like']),
      'temp_min': round(data['main']['temp_min']),
      'temp_max': round(data['main']['temp_max']),
      'humidity': round(data['main']['humidity']),
      'description': data['weather'][0].get('description', 'No description') if 'weather' in data and data['weather'] else 'No description',
      'country': data['sys'].get('country', 'N/A'),
      'wind_gust_dir': data['wind'].get('deg', 0),
      'pressure': data['main'].get('pressure', 0),
      'WindGustSpeed': data['wind'].get('speed', 0)
  }

In [30]:
def read_historical_data(filename):
  df = pd.read_csv('weather.csv')
  df = df.dropna()
  df = df.drop_duplicates()
  return df

In [ ]:
def prepare_data(data):
  le = LabelEncoder() #create a LabelEncoder instance
  data['WindGustDir'] = le.fit_transform(data['WinGustDir'])
  data['RainTomorrow'] = le.fit_transform(data['RainTomorrow'])

  #define the feature variable and target variables
  X = data[['MinTemp', 'MaxTemp', 'WindGustDir', 'WindGustSpeed', 'Pressure', 'Temp']] # feature variable
  y = data['RainTomorrow'] #target variable

  return X, y, le #return feature variable, taget variable and the label encoder

In [ ]:
def train_rain_model(X, y):
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
  model = RandomForestClassifier(n_estimators=100, random_state=42)
  model.fit(X_train, y_train) #train the model

  y_pred = model.predict(X_test) #to make prediction on test set

  print("Mean Squared Error for the Rain Model")

  print(mean_squared_error(y_test, y_pred))

  return model

In [ ]:
def prepare_regression_data(data, feature):
  X, y = [], [] #initialize list for feature and target value

  for i in range(len(data) -1):
    X.append(data[feature].iloc[1])
    y.append(data[feature].iloc[i + 1])

  X = np.array(X).reshape(-1, 1)
  y = np.array(y)

  return X, y

In [31]:
def train_regression_model(X, y):
  model = RandomForestRegressor(n_estimators=100, random_state=42)
  model.fit(X, y)
  return model

In [11]:
def predict_future(model, current_value):
  predictions = [current_value]

  for i in range(5):
    next_value = model.predict(np.array([[predictions[-1]]]))

    predictions.append(next_value[0])

  return predictions[1:]

In [ ]:
def weather_view():
    city = input('Enter any city name: ')
    current_weather = get_current_weather(city)

    # Load historical data
    historical_data = read_historical_data('../weather.csv')

    # Prepare and train the rain prediction model
    X, y, le = prepare_data(historical_data)
    rain_model = train_rain_model(X, y)

    # Map wind direction to compass points
    wind_deg = current_weather['wind_gust_dir'] % 360
    compass_points = [
        ("N", 0, 11.25), ("NNE", 11.25, 33.75), ("NE", 33.75, 56.25),
        ("ENE", 56.25, 78.75), ("E", 78.75, 101.25), ("ESE", 101.25, 123.75),
        ("SE", 123.75, 146.25), ("SSE", 146.25, 168.75), ("S", 168.75, 191.25),
        ("SSW", 191.25, 213.75), ("SW", 213.75, 236.25), ("WSW", 236.25, 258.75),
        ("W", 258.75, 281.25), ("WNW", 281.25, 303.75), ("NW", 303.75, 326.25),
        ("NNW", 326.25, 348.75)
    ]

    compass_direction = next(point for point, start, end in compass_points if start <= wind_deg < end)

    compass_direction_encoded = le.transform([compass_direction])[0] if compass_direction in le.classes_ else -1

    current_data = {
        'MinTemp': current_weather['temp_min'],
        'MaxTemp': current_weather['temp_max'],
        'WindGustDir': compass_direction_encoded,
        'WindGustSpeed': current_weather['wind_gust_speed'],
        'Humidity': current_weather['humidity'],
        'Pressure': current_weather['pressure'],
        'Temp': current_weather['current_temp'],
    }

    current_df = pd.DataFrame([current_data])

    # Rain prediction
    rain_prediction = rain_model.predict(current_df)[0]

    # Prepare regression model for temperature and humidity
    X_temp, y_temp = prepare_regression_data(historical_data, 'Temp')
    X_hum, y_hum = prepare_regression_data(historical_data, 'Humidity')

    temp_model = train_regression_model(X_temp, y_temp)
    hum_model = train_regression_model(X_hum, y_hum)

    # Predict future temperature and humidity
    future_temp = predict_future(temp_model, current_weather['temp_min'])
    future_humidity = predict_future(hum_model, current_weather['humidity'])

    # Prepare time for future predictions
    timezone = pytz.timezone('Asia/Kolkata')
    now = datetime.now(timezone)
    next_hour = now + timedelta(hours=1)
    next_hour = next_hour.replace(minute=0, second=0, microsecond=0)

    future_times = [(next_hour + timedelta(hours=i)).strftime("%H:00") for i in range(5)]

    # Display results
    print(f"City: {city}, {current_weather['country']}")
    print(f"Current Temperature: {current_weather['current_temp']}°C")
    print(f"Feels Like: {current_weather['feels_like']}°C")
    print(f"Minimum Temperature: {current_weather['temp_min']}°C")
    print(f"Maximum Temperature: {current_weather['temp_max']}°C")
    print(f"Humidity: {current_weather['humidity']}%")
    print(f"Weather Prediction: {current_weather['description']}")
    print(f"Rain Prediction: {'Yes' if rain_prediction else 'No'}")

    print("\nFuture Temperature Predictions:")
    for time, temp in zip(future_times, future_temp):
        print(f"{time}: {round(temp, 1)}°C")

    print("\nFuture Humidity Predictions:")
    for time, humidity in zip(future_times, future_humidity):
        print(f"{time}: {round(humidity, 1)}%")
weather_view()

API Response: {'cod': 401, 'message': 'Invalid API key. Please see https://openweathermap.org/faq#error401 for more info.'}
Error fetching data: Invalid API key. Please see https://openweathermap.org/faq#error401 for more info.


AttributeError: 'LabelEncoder' object has no attribute 'fit_transfrom'